In [1]:
import pandas as pd

FILE = "/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv"

data = pd.read_csv(FILE)
data = data[["id", "title", "num_points"]]

In [2]:
import numpy as np

seed = 7
np.random.seed(seed)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

title = data["title"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(title)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 87282 unique tokens.


In [4]:
GOOD_THRESHOLD = 5
MAX_SEQUENCE_LENGTH = 24

train = data.sample(frac=0.8)
test = data.drop(train.index)

In [5]:
def prepareData(df):    
    good = df[df["num_points"] >= GOOD_THRESHOLD]
    bad = df[df["num_points"] < GOOD_THRESHOLD]
    bad = bad.sample(n=good.shape[0])
    data = good.append(bad)
    data = data.sample(frac=1).reset_index(drop=True)
    
    num_points = data["num_points"].values

    y_train = np.zeros((len(num_points), 2), dtype=int)
    y_original = np.zeros((len(num_points)), dtype=int)
    for i in range(0, len(num_points)):
        y_train[i, 1] = int(num_points[i] >= GOOD_THRESHOLD)
        y_train[i, 0] = int(num_points[i] < GOOD_THRESHOLD)
        y_original[i] = int(num_points[i] >= GOOD_THRESHOLD)
        
    sequences = tokenizer.texts_to_sequences(data["title"])
    x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return x_train, y_train, y_original


In [6]:
x_full, y_full, y2_full = prepareData(data)
x_train, y_train, _ = prepareData(train)
x_test, y_test, _ = prepareData(test)

print(y2_full)

[0 1 0 ..., 1 1 0]


In [7]:
import os
import numpy as np

embeddings_index = {}
f = open(os.path.join('/Users/Shared/data/glove.6B/', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [13]:
from keras.layers import Input, Convolution1D, MaxPooling1D, Dense, Flatten, Dropout, Embedding
from keras.models import Model
from keras.regularizers import l2, activity_l2

def create_baseline():
    embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    #x = Convolution1D(16, 5, activation='relu')(embedded_sequences)
    #x = Dropout(0.5)(x)
    #x = MaxPooling1D()(x)
    #x = Convolution1D(16, 5, activation='relu')(embedded_sequences)
    #x = Dropout(0.5)(x)
    #x = MaxPooling1D()(x)
    #x = Dropout(0.5)(x)
    x = Flatten()(embedded_sequences)
    x = Dropout(0.5)(x)
    x = Dense(64, init='uniform', activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(2, activation='softmax')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc', 'precision'])
    return model

In [14]:
from sklearn.metrics import precision_score, recall_score

def validate(model, x_test, y_test):
    test_truth = np.apply_along_axis(lambda x: np.argmax(x), 1, y_test)
    test_pred = model.predict(x_test)
    test_pred = np.apply_along_axis(lambda x: np.argmax(x), 1, test_pred)
    precision = precision_score(test_truth, test_pred)
    recall = recall_score(test_truth, test_pred)
    print(precision)
    print(recall)
    return precision, recall

In [15]:
from keras.callbacks import EarlyStopping
es = EarlyStopping('val_precision', patience=1, mode='max')

In [17]:
from sklearn.model_selection import StratifiedKFold

N = 5

kfold = StratifiedKFold(n_splits=N, shuffle=True, random_state=seed)
kfold.get_n_splits(x_full, y_full)

print(y2_full)

precision = 0
recall = 0
for train_index, test_index in kfold.split(x_full, y2_full):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x_full[train_index], x_full[test_index]
    y_train, y_test = y_full[train_index], y_full[test_index]
    model = create_baseline()
    model.fit(x_train, y_train, nb_epoch=100, batch_size=128, validation_data=(x_test, y_test), callbacks=[es])
    p, r = validate(model, x_test, y_test)
    precision += p
    recall += r
    
print("Precision: %.2f" % (precision / N))
print("Recall: %.2f" % (recall / N))

[0 1 0 ..., 1 1 0]
TRAIN: [     0      3      5 ..., 136977 136978 136980] TEST: [     1      2      4 ..., 136976 136979 136981]
Train on 109584 samples, validate on 27398 samples
Epoch 1/100
109584/109584 [==============================] - 13s - loss: 0.6934 - acc: 0.5015 - precision: 0.5015 - val_loss: 0.6929 - val_acc: 0.5070 - val_precision: 0.5070
Epoch 2/100
109584/109584 [==============================] - 13s - loss: 0.6925 - acc: 0.5109 - precision: 0.5109 - val_loss: 0.6921 - val_acc: 0.5143 - val_precision: 0.5143
Epoch 3/100
109584/109584 [==============================] - 13s - loss: 0.6917 - acc: 0.5165 - precision: 0.5165 - val_loss: 0.6909 - val_acc: 0.5245 - val_precision: 0.5245
Epoch 4/100
109584/109584 [==============================] - 14s - loss: 0.6909 - acc: 0.5226 - precision: 0.5226 - val_loss: 0.6907 - val_acc: 0.5234 - val_precision: 0.5234
Epoch 5/100
109584/109584 [==============================] - 14s - loss: 0.6905 - acc: 0.5264 - precision: 0.5264 - val